In [1]:
!pip install ultralytics


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from ultralytics import YOLO
import os, shutil
from pathlib import Path
import math


In [3]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import shutil
from pathlib import Path
from ultralytics import YOLO

In [4]:
# Define input and output folders
video_folder = Path(r"C:\Users\amala\Downloads\test_videos")
alert_folder = Path(r"C:\Users\amala\Downloads\ALERTS!!")
os.makedirs(alert_folder, exist_ok=True)

In [9]:
import math
import os, shutil
import cv2
from ultralytics import YOLO

# Euclidean distance
def euclidean(p1, p2):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

# MAIN VIDEO ANALYSIS
def analyze_video(video_path, model, conf_threshold=0.3, show=True):

    results = model.track(
        source=video_path,
        conf=conf_threshold,
        persist=True,
        stream=True,
        show=False,
        verbose=False,
        tracker="bytetrack.yaml"
    )

    names = model.names
    DIST_THRESHOLD = 180

    id_class_history = {}       # unified_id -> list of (label, conf)
    id_last_position = {}       # unified_id -> (x,y)
    byteID_to_unified = {}      # YOLO tracker ID -> unified ID
    next_unified_id = 1
    id_frame_presence = {}      # unified_id -> list of frame indexes
    frame_index = 0

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    ret, frame0 = cap.read()
    if not ret:
        raise RuntimeError("Cannot read video.")
    h, w, _ = frame0.shape
    cap.release()

    # Frame LOOP
    for r in results:
        frame = r.orig_img
        boxes = r.boxes
        frame_index += 1

        if boxes is None or boxes.xyxy is None:
            continue

        labels = [names[int(c)] for c in boxes.cls]
        ids = boxes.id
        if ids is None:
            ids = [None] * len(labels)

        for label, box, conf_score, byte_id in zip(labels, boxes.xyxy, boxes.conf, ids):
            if label not in ["child", "adult"]:
             continue

            x1, y1, x2, y2 = map(int, box)
            cx = (x1 + x2) / 2
            cy = (y1 + y2) / 2

            # Assign unified ID using Euclidean distance
            if byte_id not in byteID_to_unified:
                matched = None
                for uid, last_pos in id_last_position.items():
                    if euclidean((cx, cy), last_pos) < DIST_THRESHOLD:
                        matched = uid
                        break
                if matched is None:
                    matched = next_unified_id
                    next_unified_id += 1
                byteID_to_unified[byte_id] = matched

            unified_id = byteID_to_unified[byte_id]

            # Update tracking
            id_class_history.setdefault(unified_id, []).append((label, conf_score))
            id_last_position[unified_id] = (cx, cy)
            id_frame_presence.setdefault(unified_id, []).append(frame_index)

            # Draw boxes
            if show:
                color = (0, 255, 0) if label == "child" else (0, 0, 255)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{label} {conf_score:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        if show:
            cv2.imshow("Elevator Monitoring", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cv2.destroyAllWindows()

    # FINAL CLASS PER unified_id
    final_class = {}
    for uid, history in id_class_history.items():
        score_child = sum(c for lbl, c in history if lbl == "child")
        score_adult = sum(c for lbl, c in history if lbl == "adult")
        final_class[uid] = "child" if score_child >= score_adult else "adult"

    total_children = sum(1 for v in final_class.values() if v == "child")
    total_adults = sum(1 for v in final_class.values() if v == "adult")

    # ALERT LOGIC based on full video
    child_ids = [uid for uid, cls in final_class.items() if cls == "child"]
    adult_ids = [uid for uid, cls in final_class.items() if cls == "adult"]

    alert = False

    for cid in child_ids:
        child_frames = set(id_frame_presence.get(cid, []))
        appeared_with_adult = False
        for aid in adult_ids:
            adult_frames = set(id_frame_presence.get(aid, []))
            if child_frames.intersection(adult_frames):
                appeared_with_adult = True
                break
        if not appeared_with_adult:
            alert = True
            break

    # SUMMARY
    print("\n--- Summary ---")
    print(f"Video: {os.path.basename(video_path)}")
    print(f" children detected: {total_children}")
    print(f" adults detected: {total_adults}")
    print(f"Total frames: {total_frames}")
    print("ALERT: Child alone!" if alert else "No alert triggered.")

    return {"video": os.path.basename(video_path), "alert": alert}


# PROCESS MULTIPLE VIDEOS
def process_videos(video_folder, model, conf=0.3):
    alert_folder = os.path.join(video_folder, "alert_videos")
    os.makedirs(alert_folder, exist_ok=True)

    videos = [f for f in os.listdir(video_folder)
              if f.lower().endswith((".mp4", ".avi"))]

    print(f"Processing {len(videos)} videos...\n")

    for file_name in videos:
        print(f"Analyzing video: {file_name}")
        video_path = os.path.join(video_folder, file_name)

        result = analyze_video(video_path, model, conf_threshold=conf, show=True)

        if result["alert"]:
            dest = os.path.join(alert_folder, file_name)
            shutil.copy(video_path, dest)
            print(f"Moved alert video to: {dest}\n")




In [10]:
from ultralytics import YOLO
from pathlib import Path
model_path = Path(r"C:\Users\amala\Downloads\best.pt")
model = YOLO(model_path)
process_videos(video_folder, model, conf=0.3)



Processing 9 videos...

Analyzing video: 1.mp4

--- Summary ---
Video: 1.mp4
 children detected: 1
 adults detected: 0
Total frames: 450
ALERT: Child alone!
Moved alert video to: C:\Users\amala\Downloads\test_videos\alert_videos\1.mp4

Analyzing video: 2.mp4

--- Summary ---
Video: 2.mp4
 children detected: 1
 adults detected: 0
Total frames: 450
ALERT: Child alone!
Moved alert video to: C:\Users\amala\Downloads\test_videos\alert_videos\2.mp4

Analyzing video: 232.mp4

--- Summary ---
Video: 232.mp4
 children detected: 0
 adults detected: 1
Total frames: 145
No alert triggered.
Analyzing video: 25.mp4

--- Summary ---
Video: 25.mp4
 children detected: 1
 adults detected: 1
Total frames: 225
No alert triggered.
Analyzing video: 26.mp4

--- Summary ---
Video: 26.mp4
 children detected: 1
 adults detected: 1
Total frames: 349
No alert triggered.
Analyzing video: 29.mp4

--- Summary ---
Video: 29.mp4
 children detected: 1
 adults detected: 1
Total frames: 363
No alert triggered.
Analyzing 